In [1]:
# Importar las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


In [3]:
# Cargar la imagen
image_path = '/mnt/data/Imagen_L2_ML.jpg'
image = Image.open(image_path)

# Mostrar la imagen
plt.imshow(image)
plt.axis('off')  # No mostrar los ejes
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Imagen_L2_ML.jpg'